In [1]:
import newspaper
from bs4 import BeautifulSoup
from warcio.archiveiterator import ArchiveIterator
import pandas as pd
import os
import re
# install newspaper3k, lxml[html_clean]

In [2]:
def decode_warc(path):
    pages = []

    # мусор
    excluded_strings = ["このページの本文へ", "表示形式: PC ⁄ スマホ", ""]

    with open(path, 'rb') as stream:
        for record in ArchiveIterator(stream):
            if record.rec_type == 'response':

                html_bytes = record.content_stream().read()

                encoding = None

                partial_html = html_bytes.decode('utf-8', errors='ignore')
                soup = BeautifulSoup(partial_html, 'html.parser')
                meta_tag = soup.find('meta', attrs={'charset': True})
                if meta_tag:
                    encoding = meta_tag['charset']
                else:
                    meta_tag = soup.find('meta', attrs={'http-equiv': 'Content-Type'})
                    if meta_tag and 'content' in meta_tag.attrs:
                        content_type = meta_tag['content']
                        if 'charset=' in content_type:
                            encoding = content_type.split('charset=')[-1]

                if not encoding:
                    #print("Failed to detect encoding from metadata.")
                    continue

                #print(f"Detected encoding from metadata: {encoding}")

                try:
                    html = html_bytes.decode(encoding)
                    pages.append({
                        'url': record.rec_headers.get_header('WARC-Target-URI'),
                        'html': html,
                        'text': ''
                    })
                except (UnicodeDecodeError, TypeError):
                    #print(f"Failed to decode using detected encoding: {encoding}")
                    continue
    return pages

In [3]:
def parse_site(pages):
    article = newspaper.Article(url = ' ')
    for page in pages:
        html = page['html']
        article.set_html(html)
        article.parse()
        page['text'] = article.text
    return pages

In [4]:
root_dir = r'news_dataset\ja'
count_total = 0
count = 0
skip_files = 248 
mem_thresh = 1073741824 / 4
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        count_total += 1
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        if count <= skip_files:
            count += 1
            continue

        dir_path = r''
        if re.search('2016', file):
            dir_path = r'parsed_news\2016'
        elif re.search('2017', file):
            dir_path = r'parsed_news\2017'
        elif re.search('2019', file):
            dir_path = r'parsed_news\2019'
        elif re.search('2022', file):
            dir_path = r'parsed_news\2022'
        elif re.search('2023', file):
            dir_path = r'parsed_news\2023'
        #print(subdir, file)

        source_path = subdir + '\\' + file
        size = os.path.getsize(source_path)
        if size > mem_thresh:
            print(file, ' too big, skipping')
            count += 1
            print(count, '/', count_total, ' files parsed')
            continue

        pages = decode_warc(source_path)
        print('decoded ', source_path)
        pages = parse_site(pages)

        path = dir_path + '\\' + file[:-5] +'.csv' 
        df = pd.DataFrame.from_dict(pages)
        df.to_csv(path)
        print(file[:-5], ' parsed')
        count += 1
        print(count, '/', count_total, ' files parsed')


www.ishark.com-2023-04.warc  too big, skipping
250 / 375  files parsed
www.ishark.com-2023-05.warc  too big, skipping
251 / 375  files parsed
www.iza.ne.jp-2023-03.warc  too big, skipping
252 / 375  files parsed
www.iza.ne.jp-2023-04.warc  too big, skipping
253 / 375  files parsed
www.iza.ne.jp-2023-05.warc  too big, skipping
254 / 375  files parsed
www.iza.ne.jp-2023-06.warc  too big, skipping
255 / 375  files parsed
www.iza.ne.jp-2023-07.warc  too big, skipping
256 / 375  files parsed
www.jiji.com-2023-03.warc  too big, skipping
257 / 375  files parsed
www.jiji.com-2023-04.warc  too big, skipping
258 / 375  files parsed
www.jiji.com-2023-05.warc  too big, skipping
259 / 375  files parsed
www.jiji.com-2023-06.warc  too big, skipping
260 / 375  files parsed
www.jiji.com-2023-07.warc  too big, skipping
261 / 375  files parsed
decoded  news_dataset\ja\www.jomo-news.co.jp-2016-08.warc\www.jomo-news.co.jp-2016-08.warc
www.jomo-news.co.jp-2016-08  parsed
262 / 375  files parsed
decoded  new